Data Exploration

In [2]:
import pandas as pd
import os

os.chdir("/Users/claud/OneDrive/Ambiente de Trabalho/Ciencia de dados/Projeto")

df = pd.read_csv('customer.csv')

In [6]:
df.head()

,Unnamed: 0,custid,sex,is_employed,income,marital_status,health_ins,housing_type,num_vehicles,age,state_of_res,code_column,gas_usage,rooms,recent_move_b
0,7,000006646_03,Male,True,22000.0,Never married,True,Homeowner free and clear,0.0,24,Alabama,1047,210.0,3,F
1,8,000007827_01,Female,NaN,23200.0,Divorced/Separated,True,Rented,0.0,82,Alabama,1047,3.0,6,T
2,9,000008359_04,Female,True,21000.0,Never married,True,Homeowner with mortgage/loan,2.0,31,Alabama,1047,40.0,3,F
3,10,000008529_01,Female,NaN,37770.0,Widowed,True,Homeowner free and clear,1.0,93,Alabama,1047,120.0,2,F
4,11,000008744_02,Male,True,39000.0,Divorced/Separated,True,Rented,2.0,67,Alabama,1047,3.0,2,F


In [7]:
df.dtypes

Unnamed: 0          int64
custid             object
sex                object
is_employed        object
income            float64
marital_status     object
health_ins           bool
housing_type       object
num_vehicles      float64
age                 int64
state_of_res       object
code_column         int64
gas_usage         float64
rooms               int64
recent_move_b      object
dtype: object

In [14]:
df["is_employed"].value_counts()

is_employed
True     44630
False     2313
Name: count, dtype: int64